# 导入数据

In [ ]:

!pip install transformers
!pip install torch


In [ ]:
import numpy as np
import pandas as pd
import transformers
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, classification_report,precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback, set_seed
from transformers import TrainingArguments, Trainer

In [ ]:
# 查看GPU信息
! /opt/bin/nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
balanced_df_all_cleaned = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned.csv')
# balanced_df_all_cleaned = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned_tokenization.csv')

In [ ]:
# replace NaN values with an empty string
balanced_df_all_cleaned['tweet_content'] = balanced_df_all_cleaned['tweet_content'].fillna('')


In [ ]:
df2=balanced_df_all_cleaned.copy()
# replace NaN values with an empty string
df2['tweet_content'] = df2['tweet_content'].fillna('')

In [ ]:
# after data balance
grouped_label = df2.groupby(['label', ]).size().reset_index(name='count')

In [ ]:
grouped_label

,label,count
0,0,10325
1,1,10325


In [ ]:
# choose 2000 rows to retrain the models
df_label_0 = df2[df2['label'] == 0].sample(n=1000, random_state=43)
df_label_1 = df2[df2['label'] == 1].sample(n=1000, random_state=43)

# merge two dataframes
df3 = pd.concat([df_label_0, df_label_1])


In [ ]:
# all rows
df3=df2

In [ ]:
df3.shape

(20650, 3)

In [ ]:
df3.head()

,ID,label,tweet_content
8426,24791,0,车王徐浪浪哥电影想红玩微拍暴力明天午两点半直播总发车仪式明天午两点半直播总发车仪式路走老脆弱...
10,18973,0,分享单曲喜欢原唱陈洁仪课代表估计追青刷朋友圈文案觉生畏头秃做分钟报告拍张低头发际线堪忧片放进...
10042,31405,0,魔族炼气士刚刚祭灵未扑头老龟便见道道剑气落半空中象头鹰头独眼魔神头蛛魔神头头颅雨般落选李彤拒...
6144,15303,0,卡日签领红包签越红包越限时奖励快领错惜微博红包祝生日快乐湖北黄氏考察调研记浦仲诚隐梅斋湖北黄...
3002,13335,0,轉發微博轉發微博工作王道刘昊然张慧雯林姑娘腰功厉害王源确认出演玄幻电视剧主宰男主角牧尘首位担...


In [ ]:

# Define pretrained tokenizer and model

model_name = "distilbert-base-uncased" # distilbert-base-uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
##############################
# Data: Text Input and Label #
##############################
X = list(df3["tweet_content"])
y = list(df3["label"])


In [ ]:
########################
# Create torch dataset #
########################
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
# 方法一：定义计算评价指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:

########################
# Run cross-validation #
########################
def run_cross_validation(model_name='DistilBert',
                         X=X,
                         y=y,
                         splits=5,# 默认是5
                         epoch=8,# 默认是8
                         checkpoint=False):

    kfold = StratifiedShuffleSplit(n_splits=splits, test_size=0.1, random_state=1127)# 默认test_size=0.1
    # kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=1127)
    n_fold = 1

    print("Developing Model with Cross validation for: " + model_name)
    for train, test in tqdm(kfold.split(X, y)):

        print("Running for Fold: ",n_fold)
        train_index = list(train)
        test_index = list(test)

        X_train = [X[i] for i in train_index]
        y_train = [y[i] for i in train_index]
        X_val = [X[i] for i in test_index]
        y_val = [y[i] for i in test_index]

        # Tokenize
        X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
        X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

        # Create torch dataset
        train_dataset = Dataset(X_train_tokenized, y_train)
        val_dataset = Dataset(X_val_tokenized, y_val)

        # Fine Tune Transformer
        # Define Trainer
        args = TrainingArguments(
            output_dir="content/drive/MyDrive_binary/output_" + model_name + "/fold"+str(n_fold),
            evaluation_strategy="epoch",
            save_strategy="epoch",
            #eval_steps=500,
            #per_device_train_batch_size=1,
            #per_device_eval_batch_size=1,
            num_train_epochs=epoch, #1 was okay
            seed=1127,
            load_best_model_at_end=True,
        )

        trainer = Trainer(
        #model_init=model_init,
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        )

        if n_fold <=4:# 默认是4
            checkpoint_temp=True # 当设置为 true 时，会在训练过程中生成临时检查点文件，以便在发生意外情况（如程序崩溃）时可以从最近的检查点恢复训练。
        else:
            checkpoint_temp=False

        trainer.train(resume_from_checkpoint=checkpoint)
        print("Complete for fold", n_fold)
        n_fold= n_fold + 1

In [ ]:
#####################
# Run Test Test #
#####################

# Train and Test Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=112)


In [ ]:

# Tokenize 用没有tokenized的数据集
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [ ]:
# Tokenize 用tokenized的数据集
# X_train_tokenized=X_train
# X_val_tokenized=X_test

In [ ]:

# Train and Validate Set

# Create torch dataset
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_test)

def run_hold_out_split(model_name='DistilBert',
                       epoch=8,# 默认是8
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=False):

    print("Developing Model with Hold Out Splits for: " + model_name)
    # Fine Tune Transformer
    # Define Trainer
    args = TrainingArguments(
      output_dir="content/drive/MyDrive_binary/output_" + model_name + "/holdout",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      #eval_steps=500,
      #per_device_train_batch_size=1,
      #per_device_eval_batch_size=1,
      num_train_epochs=epoch, #1 was okay
      seed=1127,
      load_best_model_at_end=True,
    )

    trainer = Trainer(
    # model_init=model_init,
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    trainer.train(resume_from_checkpoint=checkpoint)
    print("Complete for hold-out validate set")


In [ ]:
###########################
# Predict (Hold Out Test) #
###########################

from sklearn.metrics import confusion_matrix
def compute_metrics_holdout(model_name='DistilBert',
                            model_path='content/drive/MyDrive_binary/output_DistilBert/holdout/checkpoint-3820',
                            average_method='binary',
                            X_test=X_test):

    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

    # Create torch dataset
    test_dataset = Dataset(X_test_tokenized)

    # Load trained model
    model_pred = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

    # Define test trainer
    test_trainer = Trainer(model_pred)

    # Make prediction
    raw_pred, _, _ = test_trainer.predict(test_dataset)

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    # Compute metrics
    precision_cb = precision_score(y_test, y_pred, average=average_method, pos_label=1)
    recall_cb = recall_score(y_test, y_pred, average=average_method, pos_label=1)
    f1_cb = f1_score(y_test, y_pred, average=average_method, pos_label=1)

    precision_ncb = precision_score(y_test, y_pred, average=average_method, pos_label=0)
    recall_ncb = recall_score(y_test, y_pred, average=average_method, pos_label=0)
    f1_ncb = f1_score(y_test, y_pred, average=average_method, pos_label=0)

    precision_overall = precision_score(y_test, y_pred, average='macro')
    recall_overall = recall_score(y_test, y_pred, average='macro')
    f1_overall = f1_score(y_test, y_pred, average='macro')

    # Print Results
    print("Classification Report:")
    print(classification_report(y_test,y_pred))
    print()
    print("Label 1: depression")
    print("Precision: ", precision_cb)
    print("Recall: ", recall_cb)
    print("F-measure: ", f1_cb)
    print()
    print("Label 0: Non-depression")
    print("Precision: ", precision_ncb)
    print("Recall: ", recall_ncb)
    print("F-measure: ", f1_ncb)
    print()
    print("Macro Metrics")
    print("Precision: ", precision_overall)
    print("Recall: ", recall_overall)
    print("F-measure: ", f1_overall)
    print()



In [ ]:
# 第三次运行，test size=0.1, tokenization数据集, all rows
run_cross_validation(model_name='DistilBert',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilBert


0it [00:00, ?it/s]

Running for Fold:  1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.525500,0.513969,0.786925,0.794439,0.774443,0.784314
2,0.485200,0.493793,0.775303,0.728882,0.877057,0.796134
3,0.473100,0.486263,0.792736,0.844926,0.717328,0.775916
4,0.431200,0.491558,0.795157,0.810591,0.770571,0.790074
5,0.418900,0.507783,0.802906,0.794727,0.817038,0.805728
6,0.393000,0.538104,0.806295,0.831414,0.768635,0.798793


...

Complete for fold 1
Running for Fold:  2


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.476000,0.445203,0.804843,0.886029,0.699903,0.782044
2,0.464100,0.473166,0.808232,0.797386,0.826718,0.811787
3,0.450100,0.437500,0.811138,0.868270,0.733785,0.795383
4,0.429900,0.477897,0.820339,0.815238,0.828654,0.821892
5,0.414000,0.511431,0.807748,0.774138,0.869313,0.818969
6,0.400900,0.489626,0.813559,0.852174,0.758955,0.802867


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-2324
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-2324/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-2324/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-4648
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-4648/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-4648/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-6972
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold

Complete for fold 2


In [ ]:
# 第二次运行，test size=0.1, tokenization数据集, all rows
run_cross_validation(model_name='DistilBert',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilBert


0it [00:00, ?it/s]

Running for Fold:  1


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.499200,0.497938,0.797579,0.780310,0.828654,0.803756
2,0.467500,0.473515,0.785956,0.898785,0.644724,0.750846
3,0.440700,0.454828,0.811622,0.809615,0.815102,0.812349


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-2324
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-2324/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-2324/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-4648
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-4648/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-4648/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-6972
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.499200,0.497938,0.797579,0.780310,0.828654,0.803756
2,0.467500,0.473515,0.785956,0.898785,0.644724,0.750846
3,0.440700,0.454828,0.811622,0.809615,0.815102,0.812349
4,0.408700,0.475021,0.798547,0.810050,0.780252,0.794872
5,0.389300,0.556204,0.800000,0.774823,0.846079,0.808885
6,0.371800,0.557518,0.807748,0.841935,0.757986,0.797759


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-9296
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-9296/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-9296/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-11620
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-11620/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-11620/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold1/checkpoint-13944
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/

Complete for fold 1
Running for Fold:  2


In [ ]:
#holdout
run_hold_out_split(model_name='DistilBert',
                       epoch=8, # 默认 是8
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=False)

In [ ]:
# 第四次运行，test size=0.1, no tokenization数据集, all rows
run_cross_validation(model_name='DistilBert',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilBert


0it [00:00, ?it/s]

Running for Fold:  1


In [ ]:
# 第三次运行，test size=0.1, no tokenization数据集, 2000rows
run_cross_validation(model_name='DistilBert',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilBert


0it [00:00, ?it/s]

Running for Fold:  1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.687831,0.505000,0.502513,1.000000,0.668896
2,No log,0.947494,0.715000,0.921569,0.470000,0.622517
3,0.650900,0.498424,0.785000,0.851852,0.690000,0.762431
4,0.650900,0.594121,0.785000,0.761468,0.830000,0.794258
5,0.462500,0.495666,0.785000,0.787879,0.780000,0.783920
6,0.462500,0.609861,0.795000,0.904110,0.660000,0.763006
7,0.379600,0.597070,0.795000,0.831461,0.740000,0.783069
8,0.379600,0.653985,0.785000,0.793814,0.770000,0.781726


...

Complete for fold 1
Running for Fold:  2


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.582580,0.685000,1.000000,0.370000,0.540146
2,No log,0.478107,0.835000,0.971831,0.690000,0.807018
3,0.449400,0.355319,0.845000,0.810811,0.900000,0.853081
4,0.449400,0.398506,0.875000,0.941176,0.800000,0.864865
5,0.360900,0.388685,0.865000,0.929412,0.790000,0.854054
6,0.360900,0.428351,0.870000,0.902174,0.830000,0.864583


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-225
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-225/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-225/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-450
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-450/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-450/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoint-675
Configuration saved in content/drive/MyDrive_binary/output_DistilBert/fold2/checkpoi

Complete for fold 2
